In [ ]:
# -*- coding: utf-8 -*-
import keras
from keras.optimizers import SGD, Adam
from keras.initializers import RandomNormal, RandomUniform
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Concatenate, InputLayer, Input
from keras.layers import LSTM
from keras.models import load_model
import numpy as np
import pandas as pd
from module.recorder import data_loader as DL
from module.player import strategylearner as SL
import json

In [ ]:
dataset_path = "data//tessdata.csv"
df = DL.load_nim_log(dataset_path)

In [ ]:
took_molder = SL.data_molder.data_molder(df.get_max[0])
mod_molder = SL.data_molder.data_molder(df.get_max[0])

In [ ]:
#データフレームにデータを成形
df["winner_mod"] = [remain_winner%df.get_max[0] for remain_winner in df.remain_winner]
df["winner_converted"] = [took_molder(win) for win in df.win]
df["winner_mod_converted"] = [mod_molder(win+1) for win in df.winner_mod]
df["loser_mod"] = [remain_loser%df.get_max[0] for remain_loser in df.remain_loser]
df["loser_converted"] = [took_molder(lose) for lose in df.lose]
df["loser_mod_converted"] = [mod_molder(lose+1) for lose in df.loser_mod]

In [ ]:
#インプットするデータ作成
#[直前に相手がとった石の数],[直前に自分がとった石の数],[石の総数からとれる最大値で割ったあまり]
#今回使うデータが先手で勝ったものなので
data_loser = np.array([[0,0,0,0,0]]+[data for data in df.loser_converted[:-1]], dtype=float)
data_winner = np.array([[0,0,0,0,0]]+[data for data in df.winner_converted[:-1]], dtype=float)
data_winner_mod = np.array([data for data in df.winner_mod_converted[:]], dtype=float)

In [ ]:
#教師データ
labels = np.array([value for value in df.winner_converted], dtype=float)

In [ ]:
input_pre_took_myself = Input(shape=(df.get_max[0],))
input_pre_took_enemy = Input(shape=(df.get_max[0],))
input_pre_mod  = Input(shape=(df.get_max[0],))
merged_input = keras.layers.concatenate([input_pre_took_myself, input_pre_took_enemy], axis=-1)
pre_mid = Dense(df.get_max[0]*3*5, activation = "relu")(merged_input)
output = Dense(df.get_max[0], activation = "softmax")(pre_mid)
model = Model([input_pre_took_myself, input_pre_took_enemy, input_pre_mod], outputs = output)
model.compile(optimizer = "adam", loss="categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.fit([data_loser, data_winner, data_winner_mod], labels, epochs=10)

In [ ]:
model.save("model2.h5")